In [2]:
!pip install mido

    100% |████████████████████████████████| 61kB 767kB/s ta 0:00:01
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip install --upgrade pip

Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 1.3MB 344kB/s ta 0:00:01
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1


In [4]:
import mido
import pandas as pd
from mido import MidiFile

In [7]:
def midi_to_dataframe(midi_path,path_info=0):
    midi = MidiFile(midi_path)
    channel_list = list()
    note_list = list()
    vel_list = list()
    time_list = list()
    program_list = list() 
    channel_list_program = list()
    
    # for loop to run for midi channel(track), note, velocity, time -- fixed!            
    for i, track in enumerate(midi.tracks):
        for message in track:
            try:                
                channel_list.append(message.channel)
                try:
                    time_list.append(message.time)

                    try:
                        vel_list.append(message.velocity)
                    except:
                        vel_list.append(0)
                    try:
                        note_list.append(message.note)
                    except:
                        note_list.append(0)  
                except:
                    pass              
            except:
                pass   


    consolidate = pd.DataFrame({
            'Channel':channel_list,
     'Note' : note_list,
     'Velocity':vel_list,
    'Time':time_list
      })
    if path_info == 1:
        
        consolidate["MidiName"] = (str(midi_path.split("/")[-1]).split('.')[0])
        consolidate["Artist"] = (str(midi_path.split("/")[-2]))
        consolidate["Genre"] = (str(midi_path.split("/")[-3]))  
    else:
        consolidate["MidiName"] = ""
        consolidate["Artist"] = ""
        consolidate["Genre"] = ""  
        
    consolidate["Length"] = midi.length
    consolidate["Ticks_per_beat"] = midi.ticks_per_beat
    
    for i in midi.tracks[0]:
        if i.type == "set_tempo":
            consolidate["Tempo"] = i.tempo
                                   
    # for loop to run for program and channel(track)
    for i, track in enumerate(midi.tracks):
        for message in track:
            try:
                program_list.append(message.program)
                try:
                    channel_list_program.append(message.channel)
                except:
                    pass
            except:
                pass
            
    program_message = pd.DataFrame({
            'Channel':channel_list_program,
     'Program' : program_list
      })
    program_instrument = pd.merge(program_message,midi_program,left_on= "Program",right_on = "ProgramID",how = "left")

    program_instrument = program_instrument.drop("ProgramID",axis = 1)

    if 10 in program_instrument.Channel:
        x = program_instrument[program_instrument.Channel != 10]

        y = program_instrument[program_instrument.Channel == 10]

        new_y = pd.merge(y,midi_pro_10,left_on= "Program",right_on = "ProgramID",how = "left")
        new_y = new_y.drop(["Type","Instrument","ProgramID"],axis = 1)
        new_y.columns = ['Channel', 'Program', 'Type', 'Instrument'] 
        #print (new_y)
        program_instrument = x.append(new_y, ignore_index=True)

    combine = pd.merge(consolidate,program_instrument,on="Channel",how = "left")
    combine["Real_time"] = (combine.Tempo/(10**6*combine.Ticks_per_beat)*combine.Time)
    
    combine_w_note = pd.merge(combine,midi_note,left_on ="Note", right_on="Midi_NoteNum", how = "left")
    combine_w_note = combine_w_note.drop("Midi_NoteNum",axis = 1 )

    return combine_w_note





    

In [17]:
MidiFile("/Users/sunevan/Dropbox/Fall 2018/Capstone Project/Midi/Lady Gaga/Bad romance.mid")

<midi file '/Users/sunevan/Dropbox/Fall 2018/Capstone Project/Midi/Lady Gaga/Bad romance.mid' type 1, 15 tracks, 21337 messages>

In [19]:
midi_program = pd.read_csv("Midi_Program.csv")
midi_pro_10 = pd.read_csv("midi10.csv",encoding = "ISO-8859-1")
midi_note = pd.read_csv("midi_note.csv")

In [21]:
midi_to_dataframe("/Users/sunevan/Dropbox/Fall 2018/Capstone Project/Midi/Lady Gaga/Bad romance.mid").to_csv("Bad romance midi processed.csv")